## 연세대 세브란스에서 건강정보에 대한 식단 정보 가져오기

In [1]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
import re

url1 = 'https://sev.severance.healthcare/health/lifecare/nutrition/diseasediet.do?&mode=view&articleNo=108731&articleLimit=0'
url2 = 'https://sev.severance.healthcare/health/lifecare/nutrition/diseasediet.do?&mode=view&articleNo=70483&articleLimit=0'

def clean_text(element):
   text = element.get_text().strip()
   text = re.sub(r'\s+', ' ', text)
   return text

class CustomWebLoader(WebBaseLoader):
   def _clean_data(self, soup: bs4.BeautifulSoup) -> str:
       # 자바스크립트 태그 제거
       [s.decompose() for s in soup.find_all('script')]
       
       subject = soup.find('h3', class_='subject')
       title = clean_text(subject) if subject else ""
       
       fr_view = soup.find('div', class_='fr-view')
       if fr_view:
           paragraphs = [p for p in fr_view.find_all('p', recursive=False) 
                        if clean_text(p) and not p.find('img')]
           content = ' '.join(clean_text(p) for p in paragraphs)
           return f"제목: {title}\n\n본문:\n{content}"
       return ""

loader = CustomWebLoader(web_path=(url1, url2))
docs = loader.load()

with open('C:\jupyter\RecipeAI\stroke_diet.txt', 'w', encoding='utf-8') as f:
   f.write(str(docs[0]))

USER_AGENT environment variable not set, consider setting it to identify your requests.


## Beautifulsoup과 selenium 사용

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup
import time
import os

class SeveranceCrawler:
    def __init__(self):
        # Chrome 드라이버 옵션 설정
        self.options = webdriver.ChromeOptions()
        self.options.add_argument('--headless')  # 헤드리스 모드
        self.options.add_argument('--no-sandbox')
        self.options.add_argument('--disable-dev-shm-usage')
        
        # 크롬 드라이버 초기화
        self.driver = webdriver.Chrome(options=self.options)
        self.base_url = "https://sev.severance.healthcare/health/lifecare/nutrition/diseasediet.do"
        
    def get_article_links(self):
        """질환별 식단 목록 페이지에서 모든 글의 링크를 수집"""
        self.driver.get(self.base_url)
        time.sleep(2)  # 페이지 로딩 대기
        
        links = []
        try:
            # 더보기 버튼이 있는 경우 모두 클릭
            while True:
                try:
                    more_button = self.driver.find_element(By.XPATH, "/html/body/div/article/div[2]/div/div/div[3]/div/a")
                    if more_button.is_displayed():
                        more_button.click()
                        time.sleep(1)  # 로딩 대기
                    else:
                        break
                except NoSuchElementException:
                    break
                except Exception as e:
                    print(f"더보기 버튼 클릭 중 에러 발생: {str(e)}")
                    break
            
            # 썸네일 아이템들 찾기
            articles = self.driver.find_elements(By.CSS_SELECTOR, "div.thumb-item")
            
            for article in articles:
                try:
                    link = article.find_element(By.TAG_NAME, "a").get_attribute("href")
                    title = article.find_element(By.CSS_SELECTOR, "strong.subject").text
                    links.append({"title": title, "link": link})
                except NoSuchElementException:
                    continue
                    
        except Exception as e:
            print(f"링크 수집 중 에러 발생: {str(e)}")
            
        return links
        
    def get_article_content(self, url):
        """개별 글의 내용을 크롤링"""
        try:
            self.driver.get(url)
            time.sleep(2)  # 페이지 로딩 대기
            
            # article-body 클래스를 가진 div를 찾아서 내용 추출
            article_body = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.article-body"))
            )
            
            # BeautifulSoup으로 파싱
            soup = BeautifulSoup(article_body.get_attribute('innerHTML'), 'html.parser')
            
            # 텍스트 내용 추출 (HTML 태그 제거)
            content = soup.get_text(strip=True, separator='\n')
            
            return content
            
        except TimeoutException:
            print(f"페이지 로딩 시간 초과: {url}")
            return None
        except Exception as e:
            print(f"내용 크롤링 중 에러 발생: {str(e)}")
            return None
    
    def save_content(self, title, content, output_dir="crawled_data"):
        """크롤링한 내용을 파일로 저장"""
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            
        # 파일명에 사용할 수 없는 문자 제거
        safe_title = "".join(c for c in title if c.isalnum() or c in (' ', '-', '_')).rstrip()
        
        filename = os.path.join(output_dir, f"{safe_title}.txt")
        
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(f"제목: {title}\n\n")
            f.write(content)
            
        print(f"저장 완료: {filename}")
    
    def crawl_all_articles(self):
        """모든 글 크롤링"""
        links = self.get_article_links()
        
        for item in links:
            try:
                print(f"크롤링 중: {item['title']}")
                content = self.get_article_content(item['link'])
                
                if content:
                    self.save_content(item['title'], content)
                
                time.sleep(1)  # 서버 부하 방지를 위한 대기
                
            except Exception as e:
                print(f"글 크롤링 중 에러 발생: {str(e)}")
                continue
    
    def close(self):
        """브라우저 종료"""
        self.driver.quit()

# 실행 예시
if __name__ == "__main__":
    crawler = SeveranceCrawler()
    
    try:
        crawler.crawl_all_articles()
    finally:
        crawler.close()

크롤링 중: 항암치료 중 식사요법 Chemotherapy
저장 완료: crawled_data\항암치료 중 식사요법 Chemotherapy.txt
크롤링 중: 뇌졸중의 식사요법 Stroke
저장 완료: crawled_data\뇌졸중의 식사요법 Stroke.txt
크롤링 중: 역류성 식도염의 식사요법 Reflux esophagitis
저장 완료: crawled_data\역류성 식도염의 식사요법 Reflux esophagitis.txt
크롤링 중: 변비의 식사요법 Constipation
저장 완료: crawled_data\변비의 식사요법 Constipation.txt
크롤링 중: 골다공증의 식사요법 Osteoporosis
저장 완료: crawled_data\골다공증의 식사요법 Osteoporosis.txt
크롤링 중: 골다공증의 식사요법 2 Osteoporosis
저장 완료: crawled_data\골다공증의 식사요법 2 Osteoporosis.txt
크롤링 중: 당뇨병의 식사요법 Diabetes mellitus
저장 완료: crawled_data\당뇨병의 식사요법 Diabetes mellitus.txt
크롤링 중: 빈혈의 식사요법 Anemia
저장 완료: crawled_data\빈혈의 식사요법 Anemia.txt
크롤링 중: 지방간의 식사요법 Fatty liver
저장 완료: crawled_data\지방간의 식사요법 Fatty liver.txt
크롤링 중: 지방간의 식사요법 2 Fatty liver
저장 완료: crawled_data\지방간의 식사요법 2 Fatty liver.txt
크롤링 중: 이상지질혈증의 식사요법 Dyslipidemia
저장 완료: crawled_data\이상지질혈증의 식사요법 Dyslipidemia.txt
크롤링 중: 통풍의 식사요법 Gout
저장 완료: crawled_data\통풍의 식사요법 Gout.txt
크롤링 중: 고혈압의 식사요법 Hypertension
저장 완료: crawled_data\고혈압의 식사요법 Hypertension.t